In [4]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException, NoSuchElementException, ElementClickInterceptedException
import time
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# Function to collect route names and links from the current page
def collect_routes_from_page(driver):
    route_names = []
    route_links = []
    routes = driver.find_elements(By.XPATH, "//a[@class='route']")
    for route in routes:
        route_names.append(route.get_attribute('title'))
        route_links.append(route.get_attribute('href'))
    return route_names, route_links

# Function to extract bus details
def extract_bus_details(driver, route_name, wait):
    bus_details_list = []
    wait.until(EC.presence_of_element_located((By.XPATH, '//div[contains(@class, "bus-item")]')))
    bus_items = driver.find_elements(By.XPATH, '//div[contains(@class, "bus-item")]')

    for bus_item in bus_items:
        try:
            bus_name = bus_item.find_element(By.XPATH, './/div[contains(@class, "travels")]').text.strip()
        except:
            bus_name = 'N/A'

        try:
            bus_type = bus_item.find_element(By.XPATH, './/div[contains(@class, "bus-type")]').text.strip()
        except:
            bus_type = 'N/A'

        try:
            start_journey = bus_item.find_element(By.XPATH, './/div[contains(@class, "dp-time")]').text.strip()
        except:
            start_journey = 'N/A'

        try:
            end_journey = bus_item.find_element(By.XPATH, './/div[contains(@class, "bp-time")]').text.strip()
        except:
            end_journey = 'N/A'

        try:
            duration = bus_item.find_element(By.XPATH, './/div[contains(@class, "dur")]').text.strip()
        except:
            duration = 'N/A'

        try:
            price = bus_item.find_element(By.XPATH, './/div[contains(@class, "fare")]//span[contains(@class, "f-19 f-bold")]').text.strip()
        except:
            price = 'N/A'

        try:
            rating = bus_item.find_element(By.XPATH, './/div[contains(@class, "rating")]//span').text.strip()
        except:
            rating = 'N/A'

        try:
            seat_availability = bus_item.find_element(By.XPATH, './/div[contains(@class, "seat-left")]').text.strip()
        except:
            seat_availability = 'N/A'

        bus_details = {
            "Bus Name": bus_name,
            "Bus Type": bus_type,
            "Start of Journey": start_journey,
            "End of Journey": end_journey,
            "Duration": duration,
            "Price": price,
            "Star Rating": rating,
            "Seat Availability": seat_availability,
            "Route Name": route_name
        }

        bus_details_list.append(bus_details)

    return bus_details_list

# List of 10 state URLs
state_links = [
    "https://www.redbus.in/online-booking/ksrtc-kerala/?utm_source=rtchometile",
    "https://www.redbus.in/online-booking/apsrtc/?utm_source=rtchometile",
    "https://www.redbus.in/online-booking/tsrtc/?utm_source=rtchometile",
    "https://www.redbus.in/online-booking/ktcl/?utm_source=rtchometile",
    "https://www.redbus.in/online-booking/rsrtc/?utm_source=rtchometile",
    "https://www.redbus.in/online-booking/south-bengal-state-transport-corporation-sbstc/?utm_source=rtchometile",
    "https://www.redbus.in/online-booking/jksrtc",
    "https://www.redbus.in/online-booking/west-bengal-transport-corporation?utm_source=rtchometile",
    "https://www.redbus.in/online-booking/kaac-transport",
    "https://www.redbus.in/online-booking/pepsu/?utm_source=rtchometile"
]

# Initialize the Chrome driver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
wait = WebDriverWait(driver, 20)

# Container for all bus details
all_bus_details = []

# Process each state URL
for state_url in state_links:
    driver.get(state_url)

    # Wait for pagination to appear
    pagination_container = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'DC_117_paginationTable')))
    page_elements = pagination_container.find_elements(By.XPATH, '//div[contains(@class, "DC_117_pageTabs")]')
    num_pages = len(page_elements)

    all_route_names = []
    all_route_links = []

    # Iterate through each page
    for page in range(1, num_pages + 1):
        xpath_expression = f'//div[contains(@class, "DC_117_pageTabs") and contains(text(), "{page}")]'
        page_button = pagination_container.find_element(By.XPATH, xpath_expression)
        actions = ActionChains(driver)
        actions.move_to_element(page_button).perform()
        time.sleep(1)
        page_button.click()
        time.sleep(3)

        route_names, route_links = collect_routes_from_page(driver)
        all_route_names.extend(route_names)
        all_route_links.extend(route_links)

    # Process each route URL with its corresponding route name
    for url, route_name in zip(all_route_links, all_route_names):
        driver.get(url)
        driver.maximize_window()
        time.sleep(2)

        # Click all "View Buses" buttons
        try:
            view_buses_buttons = wait.until(
                EC.presence_of_all_elements_located((By.XPATH, "//div[@class='button' and contains(text(),'View Buses')]"))
            )
            for button in reversed(view_buses_buttons):
                try:
                    driver.execute_script("arguments[0].scrollIntoViewIfNeeded(true);", button)
                    time.sleep(1)
                    button.click()
                    time.sleep(2)
                except Exception as e:
                    print(f"Error clicking button: {e}")
                    continue
        except Exception as e:
            print(f"Error during 'View Buses' button processing: {e}")

        # Scroll to the bottom of the page multiple times to ensure all buses are loaded
        scroll_pause_time = 2
        last_height = driver.execute_script("return document.body.scrollHeight")
        while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(scroll_pause_time)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

        # Extract bus details after all content is loaded
        try:
            bus_details = extract_bus_details(driver, route_name, wait)
            all_bus_details.extend(bus_details)
        except Exception as e:
            print(f"Error extracting bus details for route {route_name}: {e}")

# Close the browser
driver.quit()

# Convert bus details to a DataFrame and remove duplicates
df = pd.DataFrame(all_bus_details).drop_duplicates(subset=["Bus Name", "Route Name"])

# Reset the index of the DataFrame
df.reset_index(drop=True, inplace=True)

# Save DataFrame to a CSV file
df.to_csv("bus_details.csv", index=False)

# Display the DataFrame
print(df.head())


Error during 'View Buses' button processing: Message: 

Error during 'View Buses' button processing: Message: 

Error during 'View Buses' button processing: Message: 

Error during 'View Buses' button processing: Message: 

Error during 'View Buses' button processing: Message: 

Error during 'View Buses' button processing: Message: 

Error extracting bus details for route Hyderabad to Warangal: Message: 
Stacktrace:
	GetHandleVerifier [0x00577143+25587]
	(No symbol) [0x0050A2E4]
	(No symbol) [0x00402113]
	(No symbol) [0x00446F62]
	(No symbol) [0x004471AB]
	(No symbol) [0x00487852]
	(No symbol) [0x0046ABE4]
	(No symbol) [0x00485370]
	(No symbol) [0x0046A936]
	(No symbol) [0x0043BA73]
	(No symbol) [0x0043C4CD]
	GetHandleVerifier [0x00854C63+3030803]
	GetHandleVerifier [0x008A6B99+3366473]
	GetHandleVerifier [0x006095F2+624802]
	GetHandleVerifier [0x00610E6C+655644]
	(No symbol) [0x00512C9D]
	(No symbol) [0x0050FD68]
	(No symbol) [0x0050FF05]
	(No symbol) [0x00502336]
	BaseThreadInitThunk

In [7]:
pip install mysql-connector-python


Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/14.3 MB ? eta -:--:--
   ---- ----------------------------------- 1.6/14.3 MB 10.5 MB/s eta 0:00:02
   ------------ --------------------------- 4.5/14.3 MB 12.2 MB/s eta 0:00:01
   ------------------- -------------------- 6.8/14.3 MB 12.3 MB/s eta 0:00:01
   -------------------------- ------------- 9.4/14.3 MB 12.5 MB/s eta 0:00:01
   -------------------------------- ------- 11.8/14.3 MB 12.1 MB/s eta 0:00:01
   -------------------------------------- - 13.9/14.3 MB 11.8 MB/s eta 0:00:01
   ---------------------------------------- 14.3/14.3 MB 11.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [12]:
import pandas as pd
import mysql.connector
import re

# Load the CSV into a DataFrame
df = pd.read_csv("C:/Users/Home/bus_details.csv")

# Database connection details
user = 'root'
password = 'Mithun252001@'
host = 'localhost'
database = 'capstone1_redbus'

# Connect to MySQL database
conn = mysql.connector.connect(user=user, password=password, host=host, database=database)
cursor = conn.cursor()

# Create the table if it doesn't exist
create_table_query = """
CREATE TABLE IF NOT EXISTS Bus_Details(
    Bus_Name VARCHAR(100),
    Bus_Type VARCHAR(100),
    Start_of_Journey VARCHAR(100),
    End_of_Journey VARCHAR(100),
    Duration VARCHAR(100),
    Price INT,
    Star_Rating FLOAT,
    Seat_Availability VARCHAR(100),
    Route_Name VARCHAR(100)
)
"""
cursor.execute(create_table_query)

# Insert data into the table
for index, row in df.iterrows():
    try:
        insert_query = """
        INSERT INTO Bus_Details (
            Bus_Name, Bus_Type, Start_of_Journey, End_of_Journey, Duration, Price, Star_Rating, Seat_Availability, Route_Name
        ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
        """
        # Check if "Price" is a string before using re.sub, or handle it if it's already numeric
        price_value = row["Price"]

        if isinstance(price_value, str):
            price = int(re.sub(r'[^\d]', '', price_value)) if price_value not in ['N/A', ''] else None
        elif isinstance(price_value, float) or isinstance(price_value, int):
            price = int(price_value)  # Convert float to int if it’s numeric
        else:
            price = None

        # Handle "Star Rating" column similarly
        star_rating = float(row["Star Rating"]) if row["Star Rating"] not in ['N/A', ''] else None

        # Prepare the values for insertion
        values = (
            row["Bus Name"],
            row["Bus Type"],
            row["Start of Journey"],
            row["End of Journey"],
            row["Duration"],
            price,
            star_rating,
            row["Seat Availability"],
            row["Route Name"]
        )

        # Execute the insert query
        cursor.execute(insert_query, values)

    except Exception as e:
        print(f"Error inserting row {index}: {e}")
        continue

# Commit the changes to the database
conn.commit()

# Close the cursor and connection
cursor.close()
conn.close()

print("Data inserted successfully.")


Data inserted successfully.
